# Wildfire Size Prediction

Predicting wildfire sizes for US states from 2011 to 2015

In [53]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, HuberRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, RegressorMixin
import os


In [54]:
class BaseModel:
    def __init__(self, config=None):
        self.config = config or {}
        self.model = self._create_model()
    
    def _create_model(self):
        raise NotImplementedError
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict(self, X):
        return self.model.predict(X)
    
    def save(self, path):
        joblib.dump(self.model, path)

    def evaluate(self, X, y_true):
        y_pred = self.predict(X)
        N = len(y_true)
        
        # Add small epsilon to avoid log(0)
        epsilon = 1e-10
        y_pred = np.maximum(y_pred, epsilon)
        y_true = np.maximum(y_true, epsilon)
        
        log_array = np.abs(np.log(y_pred/y_true))
        constrained_log_array = np.minimum(log_array, 10)  # Changed min to minimum
        sum_logs = np.sum(constrained_log_array)
        
        return -1 * (1/N * sum_logs)  # Negative because we want to maximize
    
    @classmethod
    def load(cls, path):
        instance = cls()
        instance.model = joblib.load(path)
        return instance


class RandomForest(BaseModel):
    def _create_model(self):
        params = self.config.get('model_params', {})
        return RandomForestClassifier(**params)


class AdaBoost(BaseModel):
    def _create_model(self):
        params = self.config.get('model_params', {})
        return AdaBoostClassifier(**params)


class NeuralNetwork(BaseModel):
    def _create_model(self):
        params = self.config.get('model_params', {})
        return MLPClassifier(**params)

class ModelWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y):
        # Convert to numpy array if needed
        X_arr = X.to_numpy() if hasattr(X, 'to_numpy') else X
        self.estimator.fit(X_arr, y)
        return self

    def predict(self, X):
        # Convert to numpy array if needed
        X_arr = X.to_numpy() if hasattr(X, 'to_numpy') else X
        return self.estimator.predict(X_arr)

    def get_params(self, deep=True):
        return {"estimator": self.estimator}

    def set_params(self, **params):
        self.estimator = params["estimator"]
        return self

class WildfirePredictor(BaseModel):
    MODELS = {
        'rf': RandomForestRegressor,
        'gb': GradientBoostingRegressor,
        'ada': AdaBoostRegressor,
        'elastic': ElasticNet,
        'xgb': XGBRegressor,
        'lgbm': LGBMRegressor,  # Use direct LGBMRegressor
        'catboost': CatBoostRegressor,
        'ext': ExtraTreesRegressor,
        'svr': SVR,
        'mlp': MLPRegressor,
        'huber': HuberRegressor,
        'ridge': Ridge,
        'lasso': Lasso
    }

    def _create_model(self):
        params = self.config.get('model_params', {})
        model_type = self.config.get('model_type', 'rf')
        model_class = self.MODELS.get(model_type, RandomForestRegressor)
        base_model = model_class(**params)
        return ModelWrapper(base_model)

In [55]:
def create_submission(predictions_df, output_path='submissions/submission.csv'):
    """
    Create submission file from predictions DataFrame
    
    Args:
        predictions_df: DataFrame with columns ['STATE', 'month', 'total_fire_size']
        output_path: Path to save submission file
    """
    # Ensure directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Load zero submission template
    submission = pd.read_csv('data/zero_submission.csv')
    
    # Merge predictions with template
    submission = submission[['ID', 'STATE', 'month']].merge(
        predictions_df,
        on=['STATE', 'month'],
        how='left'
    )
    
    submission = submission[['ID', 'STATE', 'month', 'total_fire_size']]

    # Save submission file
    submission.to_csv(output_path, index=False)
    print(f"Submission saved to {output_path}")

In [56]:
class WildfireData:
    def __init__(self, fire_data_path, state_data_path, weather_data_path, coordinates_path, zero_submission_path):
        self.scaler = StandardScaler()
        self.data, self.X_train, self.X_val, self.X_test, self.y_train, self.y_val = None, None, None, None, None, None

        self.load_data(fire_data_path, state_data_path, weather_data_path, coordinates_path, zero_submission_path)
        self.target_col = 'total_fire_size'
        self.features = self.data.columns.drop(self.target_col)

    def filter_features(self, features):
        self.X_train = self.X_train.filter(items=features)
        self.X_val = self.X_val.filter(items=features)
        self.X_test = self.X_test.filter(items=features)

    def load_data(self, fire_data_path, state_data_path, weather_data_path, coordinates_path, zero_submission_path):
        # Load datasets
        fire_df = pd.read_csv(fire_data_path)
        state_df = pd.read_csv(state_data_path)
        weather_df = pd.read_csv(weather_data_path)
        coordinates_df = pd.read_csv(coordinates_path)
        self.zero_submission_path = zero_submission_path
        
        # Clean up coordinates data
        coordinates_df = coordinates_df[['state&teritory', 'latitude', 'longitude']].rename(columns={'state&teritory': 'State'})
        
        # Remove any existing missing or zero rows from fire_df
        fire_df = fire_df[fire_df['total_fire_size'] > 0]
        
        # Merge datasets with filled data
        state_df = pd.merge(state_df, coordinates_df, on='State', how='left')
        self.data = combine_data(states_df=state_df, weather_df=weather_df, target_df=fire_df)

    

    def prepare_data(self, val_size=0.2):
        # Convert percentage strings to floats
        self.data['Percentage of Federal Land'] = self.data['Percentage of Federal Land'].str.rstrip('%').astype(float) / 100
        self.data['Urbanization Rate (%)'] = self.data['Urbanization Rate (%)'].astype(float) / 100
        
        # First create train set from existing data
        train_set = self.data[self.data[self.target_col].notna()].copy()
        print(f"Training set size before split: {len(train_set)}")
        print(f"Training target stats:\n{train_set[self.target_col].describe()}")
        
        # Create test set from submission template
        zero_submission = pd.read_csv(self.zero_submission_path)
        test_set = zero_submission[['STATE', 'month']].rename(columns={'STATE': 'State'})
        test_set['year_month'] = test_set['month']  # Set year_month for merging
        
        # Add state features
        state_columns = ['mean_elevation', 'Land Area (sq mi)', 'Water Area (sq mi)', 
                        'Percentage of Federal Land', 'Urbanization Rate (%)', 
                        'latitude', 'longitude']
        state_data = self.data[['State'] + state_columns].drop_duplicates('State')
        test_set = pd.merge(test_set, state_data, on='State', how='left')
        
        # Add weather features
        weather_cols = ['PRCP', 'EVAP', 'TMIN', 'TMAX']
        weather_data = self.data[['State', 'year_month'] + weather_cols].copy()
        test_set = pd.merge(
            test_set,
            weather_data,
            left_on=['State', 'year_month'],
            right_on=['State', 'year_month'],
            how='left'
        )
        
        # Add time-based features to both sets
        for df in [train_set, test_set]:
            df = self.add_month_feature(df)
            df = self.add_season_feature(df)
        
        # Fill missing weather values in test set
        for col in weather_cols:
            # Calculate averages by state and season
            avg_by_state_season = train_set.groupby(['State', 'season'])[col].mean()
            avg_by_state = train_set.groupby('State')[col].mean()
            
            # Fill nulls with state-season average, then state average
            for state in test_set['State'].unique():
                state_mask = test_set['State'] == state
                for season in test_set.loc[state_mask, 'season'].unique():
                    mask = state_mask & (test_set['season'] == season)
                    try:
                        fill_val = avg_by_state_season.loc[(state, season)]
                    except:
                        fill_val = avg_by_state.loc[state]
                    test_set.loc[mask & test_set[col].isna(), col] = fill_val
        
        # Create final splits
        self.X_train = train_set.drop(columns=[self.target_col]).reset_index(drop=True)
        self.y_train = train_set[self.target_col].values.ravel()
        
        self.X_test = test_set.reset_index(drop=True)
        self.X_test = self.X_test.sort_values(['State', 'month']).reset_index(drop=True)
        
        # Split train & validation
        self.X_train, self.X_val, self.y_train, self.y_val = split_data(
            self.X_train, self.y_train, test_size=val_size, random=False
        )
        
        print("\nDataset splits:")
        print(f"Training samples: {len(self.X_train)}")
        print(f"Validation samples: {len(self.X_val)}")
        print(f"Test samples: {len(self.X_test)}")
        print("\nFeature statistics:")
        print(self.X_test.describe())
        
        # Verify no missing values
        missing = self.X_test.isna().sum()
        if missing.any():
            print("\nMissing values in test set:")
            print(missing[missing > 0])

    def preprocess_data(self):
        # Scale features
        self.scaler = self.scaler.fit(self.X_train)
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        self.X_val = self.scaler.transform(self.X_val)

        # Set the type to np.float32
        self.X_train = self.X_train.astype(np.float32)
        self.X_train = self.X_val.astype(np.float32)
        self.X_train = self.X_test.astype(np.float32)

    def add_month_feature(self, data): 
        dates = pd.to_datetime(data['year_month'])
        data['month_since_epoch'] = ((dates.dt.year - 1970) * 12 + dates.dt.month - 1).astype(int)
        return data
    
    def add_season_feature(self, data): 
        if 'month_in_year' not in data.columns:
            data['month_in_year'] = pd.to_datetime(data['year_month']).dt.month
        
        conditions = [
            data['month_in_year'].isin([12, 1, 2]),
            data['month_in_year'].isin([3, 4, 5]),
            data['month_in_year'].isin([6, 7, 8]),
            data['month_in_year'].isin([9, 10, 11])
        ]
        choices = ['winter', 'spring', 'summer', 'fall']
        data['season'] = np.select(conditions, choices, default='unknown')
        return data

def split_data(X, y, test_size=0.2, random=False):
    if random: 
        return train_test_split(X, y, test_size=test_size)
    else: 
        return train_test_split(X, y, test_size=test_size, random_state=42)
    

def combine_data(states_df, weather_df, target_df, how='left'):
    target_df = target_df.rename(columns={'STATE': 'State', 'month': 'year_month'})
    combined_df = pd.merge(weather_df, states_df, on='State', how='right')
    combined_df = pd.merge(combined_df, target_df, on=['State', 'year_month'], how=how)
    return combined_df


In [57]:
class FeatureNamePreserver(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_names = None
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        if hasattr(X, 'columns'):
            self.feature_names = X.columns.tolist()
            return X.to_numpy()
        return X
    
    def get_feature_names_out(self, input_features=None):
        return self.feature_names

def train_and_evaluate(data_processor, model_config):
    predictor = WildfirePredictor(model_config)
    
    # Create pipeline
    numerical_features = data_processor.X_train.select_dtypes(include=[np.number]).columns
    categorical_features = data_processor.X_train.columns.drop(numerical_features)
    
    num_pipeline = Pipeline([
        ('preserves_names', FeatureNamePreserver()),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    cat_pipeline = Pipeline([
        ('preserves_names', FeatureNamePreserver()),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(sparse_output=False))
    ])

    transform_pipeline = ColumnTransformer(
        [
            ('num', num_pipeline, numerical_features),
            ('cat', cat_pipeline, categorical_features)
        ],
        verbose_feature_names_out=False
    )

    # Fit transform pipeline first
    X_train_transformed = transform_pipeline.fit_transform(data_processor.X_train)
    X_val_transformed = transform_pipeline.transform(data_processor.X_val)
    
    # Train model separately
    predictor.model.fit(X_train_transformed, data_processor.y_train)
    
    # Make predictions
    train_score = calculate_custom_score(
        predictor.model.predict(X_train_transformed),
        data_processor.y_train
    )
    val_score = calculate_custom_score(
        predictor.model.predict(X_val_transformed),
        data_processor.y_val
    )
    
    # Create final pipeline
    full_pipeline = Pipeline([
        ('preprocessor', transform_pipeline),
        ('model', predictor.model)
    ])
    
    return full_pipeline, train_score, val_score, transform_pipeline.get_feature_names_out()

def calculate_custom_score(y_pred, y_true):
    # Add small epsilon to avoid log(0)
    epsilon = 1e-10
    y_pred = np.maximum(y_pred, epsilon)
    y_true = np.maximum(y_true, epsilon)
    
    log_array = np.abs(np.log(y_pred/y_true))
    constrained_log_array = np.minimum(log_array, 10)
    sum_logs = np.sum(constrained_log_array)
    
    return -(1/len(y_true) * sum_logs)  # Negative because we want to maximize

def create_stacking_model():
    estimators = [
        ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
        ('xgb', XGBRegressor(n_estimators=100, random_state=42))
    ]
    return StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(),
        cv=5
    )

In [58]:
models_config = {
    'random_forest': {
        'model_type': 'rf',
        'model_params': {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'random_state': 42
        }
    },
    'gradient_boost': {
        'model_type': 'gb',
        'model_params': {
            'n_estimators': 150,
            'learning_rate': 0.05,
            'max_depth': 8,
            'subsample': 0.8,
            'random_state': 42
        }
    },
    'xgboost': {
        'model_type': 'xgb',
        'model_params': {
            'n_estimators': 150,
            'learning_rate': 0.05,
            'max_depth': 8,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
    },
    'lightgbm': {
        'model_type': 'lgbm',
        'model_params': {
            'n_estimators': 150,
            'learning_rate': 0.05,
            'num_leaves': 32,
            'subsample': 0.8,
            'random_state': 42
        }
    },
    'catboost': {
        'model_type': 'catboost',
        'model_params': {
            'iterations': 150,
            'learning_rate': 0.05,
            'depth': 8,
            'random_state': 42,
            'verbose': False
        }
    },
    'extra_trees': {
        'model_type': 'ext',
        'model_params': {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'random_state': 42
        }
    },
    'neural_net': {
        'model_type': 'mlp',
        'model_params': {
            'hidden_layer_sizes': (100, 50, 25),
            'activation': 'relu',
            'max_iter': 1000,  # Increased from 500
            'early_stopping': True,  # Added early stopping
            'random_state': 42
        }
    },
    'robust_regression': {
        'model_type': 'huber',
        'model_params': {
            'epsilon': 1.35,
            'max_iter': 200,
            'alpha': 0.0001
        }
    }
}

In [59]:
# Initialize data processor
data_processor = WildfireData(
    fire_data_path='data/wildfire_sizes_before_2010.csv',
    state_data_path='data/merged_state_data.csv',
    weather_data_path='data/weather_monthly_state_aggregates.csv',
    coordinates_path='data/state_coordinates.csv', 
    zero_submission_path='data/zero_submission.csv'
)

# Prepare data
data_processor.prepare_data()

X_test_original = data_processor.X_test.copy()
print(X_test_original.head())

data_processor.filter_features(['PRCP', 'EVAP', 'TMIN', 'TMAX', 'mean_elevation', 
                              'Land Area (sq mi)', 'Water Area (sq mi)', 
                              'Percentage of Federal Land', 'Urbanization Rate (%)', 
                              'latitude', 'longitude'])

# Try all models
results = {}
best_model = None
best_score = -np.inf
feature_names = None

for model_name, config in models_config.items():
    print(f"\nTraining {model_name}...")
    pipeline, train_score, val_score, feat_names = train_and_evaluate(data_processor, config)
    feature_names = feat_names  # Save feature names for later use
    
    results[model_name] = {
        'train_score': train_score,
        'val_score': val_score,
        'pipeline': pipeline
    }
    
    if val_score > best_score:
        best_score = val_score
        best_model = pipeline

# Print results with better formatting
print("\nModel Comparison (Custom Evaluation Metric):")
for model_name, scores in results.items():
    print(f"\n{model_name}:")
    print(f"Train error: {-scores['train_score']:.4f}")
    print(f"Validation error: {-scores['val_score']:.4f}")
    print(f"(Lower error is better)")

# Use best model for predictions with numpy arrays
print(f"\nUsing best model for predictions...")
X_test_transformed = best_model.named_steps['preprocessor'].transform(data_processor.X_test)
y_pred = best_model.named_steps['model'].predict(X_test_transformed)

# Create submission DataFrame with predictions
submission_df = pd.DataFrame({
    'STATE': X_test_original['State'],
    'month': X_test_original['month'],
    'total_fire_size': y_pred
})

create_submission(submission_df)

Training set size before split: 6583
Training target stats:
count    6.583000e+03
mean     1.440515e+04
std      9.878081e+04
min      1.000000e-02
25%      5.045000e+01
50%      5.350000e+02
75%      3.683100e+03
max      4.779145e+06
Name: total_fire_size, dtype: float64

Dataset splits:
Training samples: 5266
Validation samples: 1317
Test samples: 3000

Feature statistics:
       mean_elevation  Land Area (sq mi)  Water Area (sq mi)  \
count     3000.000000        3000.000000         3000.000000   
mean       518.060000       70636.920000         5296.680000   
std        521.427047       84967.355216        14115.137803   
min         20.000000        1034.000000          192.000000   
25%        180.000000       35826.000000          701.000000   
50%        300.000000       53891.500000         1501.500000   
75%        610.000000       81759.000000         4509.000000   
max       2070.000000      570641.000000        94743.000000   

       Percentage of Federal Land  Urbanizat

/home/pawel4388/ML/hackathon/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/pawel4388/ML/hackathon/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



Training extra_trees...

Training neural_net...

Training robust_regression...

Model Comparison (Custom Evaluation Metric):

random_forest:
Train error: 1.8065
Validation error: 2.1777
(Lower error is better)

gradient_boost:
Train error: 2.6208
Validation error: 2.9330
(Lower error is better)

xgboost:
Train error: 2.7008
Validation error: 3.0123
(Lower error is better)

lightgbm:
Train error: 3.5316
Validation error: 3.5695
(Lower error is better)

catboost:
Train error: 3.3360
Validation error: 3.3624
(Lower error is better)

extra_trees:
Train error: 1.6544
Validation error: 2.0570
(Lower error is better)

neural_net:
Train error: 3.2169
Validation error: 3.2023
(Lower error is better)

robust_regression:
Train error: 3.1334
Validation error: 2.9362
(Lower error is better)

Using best model for predictions...
Submission saved to submissions/submission.csv
